# Hidden Markov Model (HMM) Workshop Part 2
## Sara Carioscia and Dylan Taylor
### Hosted by Agara Bio
### February 7, 2021

## Step 1: Getting started

First, we `import` any external packages we need. Today, the only external package we use is `numpy`, which lets us use and manipulate the `array` data structure.

In [ ]:
import numpy

We wanted to make the training data more accessible to you, but unfortunately, the files were a bit too big for github in that format. As such, we've stored the training data as arrays, encoding each nucleotide and state as an integer. Each row of the array is a sequence, and each value in the row is a single nucleotide. You'll load these arrays straight into numpy arrays, and then count the emission, transition, and start probabilities.

It's also actually useful to have us encode the states and observations as integers so that we can use them to index arrays. Numpy arrays don't have row and column names, so if you want to look for the transition probability from `intergenic` to `start1`, for example, you'll have to find the index in the array that corresponds to both of those states. If we encode `intergenic` as `0` and `start1` as `1`, to find that transition probability, we just check the array at position `[0,1]`

You'll see how that works below.

Below, we've made two dictionaries, which we can use to convert from DNA bases and states into integers. We're not actually going to use this, but it shows you how the training data has been encoded.

In [ ]:
get_nuc_index = {
    'A' : 0,
    'C' : 1,
    'G' : 2,
    'T' : 3
}

get_state_index = {
    'intergenic' : 0,
    'start1' : 1,
    'start2' : 2,
    'start3' : 3,
    'exon1' : 4,
    'exon2' : 5,
    'exon3' : 6,
    'intron1' : 7,
    'intron2' : 8,
    'intron3' : 9,
    'stop1' : 10,
    'stop2' : 11,
    'stop3' : 12
}

## Step 2: Read in the data

Now that we've learned how the training data has been encoded, we want to read in our data.

We've put the training data in two `.npy` files in the same directory as this notebook:
DNA sequence training data: `HMM_DNA_training.npy`
State sequence training data: `HMM_State_training.npy`

We can use the function `numpy.load()` to load these files directly into numpy arrays.

In [ ]:
DNA_training_data = numpy.load('HMM_DNA_training.npy')
State_training_data = numpy.load('HMM_State_training.npy')

# Are the arrays the same size?
# What does the shape mean?
print(DNA_training_data.shape)
print(State_training_data.shape)

## Step 3: Learn training values to use in our model

Now that we have the data read into arrays, we want to use this training data to estimate the probabilities necessary for our model: the start, emission, and transition probabilities.

Last week, we determined the emission probabilities for each State by counting the nucleotides (how many times the State emitted an 'A', 'C', 'G' or 'T', divided by the sum for each State). We do the same thing here.

![Inferring Probabilities](inferring_probabilities.png)

First, let's talk about why we encoded our emissions and States as integers.

We have 13 states and 4 observations. We can make a 13x4 array, such that the value at position **[i,j]** in the array is the number of times the **ith** state emitted the **jth** observation. Above, we encoded the state `'exon1'` as `4` and we encoded the observation `'T'` as `3`.

So to find the number of times the `'exon1'` state emitted the nucleotide `'T'`, we would simply check the array at position `[4,3]`.

To get those counts, we make an empty array with the correct dimensions (number of rows and columns), which we fill using our data.

We make an array of zeros using the `numpy.zeros()` command, and give it the shape we want (rows,columns).

In [ ]:
# Make an array of zeros with 13 rows and 4 columns, and name it `emission_counts`
# Each row is a State, and each column shows the number of emissions of a 
# certain nucleotide for that State
emission_counts = numpy.zeros((13,4))

For the transition counts, we can create a 13x13 array, such that the value at position **[i,j]** is the number of times that State **i** transitions to State **j**.

In [ ]:
transition_counts = numpy.zeros((13,13))

Finally, we can create a one-dimensional array of size 13, that contains the "start" probabilities of each State.

In [ ]:
start_counts = numpy.zeros((13))

Now that we've created our zero-count arrays, we need to fill them using our training data.

A great way to iterate through data is to use a `for` loop. In this case, we want to investigate the State data and the DNA data at the same time. 

Using our State data array and our DNA data array, we can find the state that emitted the nucleotide at position **[i,j]** in the DNA data array by looking at the same position, **[i,j]** in the State data array.

Because of this, it's better to loop through the indicies of the arrays, rather than the rows themselves. (This allows us to use the same indicies on both arrays).

In [ ]:
# We find the number of rows in the training data using .shape[0].
# Using the `range` function with a for loop, we iterate through the row indicies of our two arrays.
for row_num in range(DNA_training_data.shape[0]): # when range() only has one input, it defaults to being the end, with start as zero and step as 1
    # We can do a similar thing for the columns, using .shape[1].
    for col_num in range(DNA_training_data.shape[1]):
        # To make things easier, let's just store the state a given position,
        # [row_num, col_num], in the State array as a variable.
        state = State_training_data[row_num,col_num]
        # We store the corresponding observation in its own variable.
        nucleotide = DNA_training_data[row_num,col_num]

        # Now that we have the State and observation,
        # we go to the corresponding spot in the emission_counts array,
        # and add 1. We do this each time we encounter that State paired with that 
        # observation, counting the total number of emissions of that observation from 
        # that state. 
        emission_counts[state,nucleotide] += 1

        # Next we need to consider the State transitions.
        # There's one fewer transition than observation in each sequence, 
        # so we need to check that we're not in the last column.
        if col_num < State_training_data.shape[1]-1:
            # We determine the next State by checking the next column in the State array. 
            # We then store that state.
            next_state = State_training_data[row_num,col_num+1]

            # We had the current state, and we just found the next state. 
            # We now go to the correct position in the `transition_counts` array and add 1.
            transition_counts[state, next_state] += 1

        # There's one last thing we want to check. If we're in the first
        # column, that State is the starting State.
        if col_num == 0:
            # We already have the current state, so we can
            # just go to that position in the start_counts array and add 1.
            start_counts[state] += 1

We had some issues with this function taking too long to run, so if this is the case for you, stop the code cell, and just load in the counts by running the cell below

In [ ]:
emission_counts = numpy.load('emission_counts.npy')
transition_counts = numpy.load('transition_counts.npy')
start_counts = numpy.load('start_counts.npy')

### Convert emission counts to probabilities 

So we now have the *counts* for each emission and transition, but we need to convert these to *probabilities*

Let's start with the emission probabilities. As before, we want to make a zero array of the correct shape (which is just the shape of the count array)

In [ ]:
# First, we make an array of zeros in the same shape as our emission counts.
# This gives us an easy place (with the correct number of rows and columns)
# to store our emission probabilities once we have them.
emission_probs = numpy.zeros(emission_counts.shape)

# For each row
for row_num in range(emission_counts.shape[0]):
    # Get the total number of emissions in the current row. 
    # We're using the `sum` command within the numpy package.
    row_sum = numpy.sum(emission_counts[row_num])
    # Make sure the sum isn't zero,
    if row_sum != 0:
        # and set the corresponding row in the probability array to the count row divided by the row total
        emission_probs[row_num] = emission_counts[row_num]/row_sum

### Convert transition counts to probabilities 

Now let's do the transition probabilities

In [ ]:
transition_probs = numpy.zeros(transition_counts.shape)

for row_num in range(transition_counts.shape[0]):
    row_sum = numpy.sum(transition_counts[row_num])
    if row_sum != 0:
        transition_probs[row_num] = transition_counts[row_num]/row_sum

### Convert start counts to probabilities 

In [ ]:
start_probs = start_counts / numpy.sum(start_counts)

## Step 4: The Forward Algorithm

Now we define a function that will use our state probabilities (`s_probs`), transition probabilities (`t_probs`), and emission probabilities (`e_probs`) to run the Forward Algorithm on a test encoded DNA sequence (`encoded_DNA_seq`). Remember that the Forward algorithm gives us the total probability that our model generated our test DNA sequence.

![Forward Algorithm](forward_alg_graphic.png)

In [ ]:
# The inputs to our function are the three probabilities, as well as an encoded DNA sequence,
# This is the output of our above function, `encode_DNA`
def forward_alg(s_probs, t_probs, e_probs, encoded_DNA_seq):
    
    # To determine the length of our DNA sequence, we take do the `shape` command to get the 
    # dimensions of our encoded DNA. This gives us (dimension1,dimension2). 
    # By taking the 0th term of that output, we get the dimension1, which in this case is the length of the DNA. 
    DNA_length = encoded_DNA_seq.shape[0]
    # We do the same thing to get the number of possible states. 
    num_states = s_probs.shape[0]
    
    # We make an empty matrix to our probabilities at each step.
    # (Think of that rectangle chart used in Week 1, where we have some
    # probability for each position)
    probability_matrix = numpy.zeros((num_states,DNA_length))
    
    # Compute the probability matrix.
    # `position` refers to each step, as we go from the beginning of `DNA_length` to the end. 
    for position in range(DNA_length):
        # Name the variable `nucleotide` to be the value at the given position in our 
        # encoded DNA sequence.
        nucleotide = encoded_DNA_seq[position]
        # If we're at the first position, we don't care about transition probability, but instead the start prob 
        if position == 0:
            # For each state in the series of states
            for state in range(num_states):
                # For the state, fill in our empty probability_matrix in the correct cell with 
                # the correct probability: the probability of starting in that state times
                # the probability of emitting that nucleotide if in that state.
                probability_matrix[state, position] = s_probs[state] * e_probs[state,nucleotide]
        # If we're at any position besides the first, we care about the transition probability
        else:
            # Consider the current state,
            for current_state in range(num_states):
                # We walk through all the states, as these are the possible previous states
                for previous_state in range(num_states):
                    # Compute the probability of the path of interest: the probability from the previous cell times
                    # the probability of getting to current state from this previous state
                    # times the emission probability of the nucleotide in the current state
                    path_prob = probability_matrix[previous_state,position-1] * t_probs[previous_state, current_state] *  e_probs[current_state, nucleotide]
                    # Because we're going to be summing the probabilities of all possible paths up until this point,
                    # we just add the calculated probability to the current probability at the current state
                    # and position in our probability matrix
                    probability_matrix[current_state, position] += path_prob
    
    # Now that we've filled out our probability matrix, we want the overall
    # probability of the model generating our sequence. To do this, we can just
    # sum the values of the last column.
    total_prob = numpy.sum(probability_matrix[:,-1]) # The first `:` means all rows, the -1 means just the last column.
    
    return total_prob

Okay! Now that our function is done, let's load in our test dna sequence. This test sequence is from an exon in a real human gene, so we would hope that our model can figure out that this sequence is from a genic region. The sequence has already been encoded using the dictionary above.

In [ ]:
test_seq = numpy.load('DNA_test_sequence.npy')

Let's call our function with the start probabilities, transitition probabilities, and emission probabilities to see what the overall probability that our model produced this sequence is!

In [ ]:
full_model_test_prob = forward_alg(start_probs, transition_probs, emission_probs, test_seq)
print(full_model_test_prob)

## Step 5: Comparing to a null model

Okay, so what does this mean? Remember, we need to compare this probability to the probability yieled by a null model to make any real conclusions. A null model would be one where only the intergenic state exists. Let's make one.

In [ ]:
# There's only one state in our null model,
# so the probability of starting in this state is 1
null_start_probs = numpy.array([1])

# Again, there's only one state in our null model,
# so there's only one transition, from 0 to 0, and
# its probability is 1
null_transition_probs = numpy.array([[1]]) # We need this array to be 2-dimensional

# For the emission probabilities, we can just pull
# these from the full model for the intergenic state
null_emission_probs = emission_probs[0].reshape((1,4)) # We needed to reshape it, so that it is also 2-dimensional

Cool! Let's run the forward algorithm, calling our new null probabilities to determine the probability that our test sequence was generated from a null, intergenic-only model

In [ ]:
null_model_test_prob = forward_alg(null_start_probs, null_transition_probs, null_emission_probs, test_seq)

print(null_model_test_prob)

The probability the sequence was generated by the full model is higher than the probability it was generated by our null model, indicating that this was probably pulled from a genic region. If we want to try to determine the state path through this region, we can use the Viterbi algorithm.

For now, let's calculate the log-odds.

In [ ]:
log_odds = numpy.log(full_model_test_prob/null_model_test_prob)
print(log_odds)

# Bonus: The Viterbi Algorithm

In [ ]:
# The inputs to our function are the three probabilities, as well as an encoded DNA sequence,
# This is the output of our above function, `encode_DNA`. 
def viterbi(s_probs, t_probs, e_probs, encoded_DNA_seq):

    # To determine the length of our DNA sequence, we take do the `shape` command to get the 
    # dimensions of our encoded DNA. This gives us (dimension1,dimension2). 
    # By taking the 0th term of that output, we get the dimension1, which in this case is the length of the DNA. 
    DNA_length = encoded_DNA_seq.shape[0]
    # We do the same thing to get the number of possible states. 
    num_states = s_probs.shape[0]

    # We make an empty matrix to store the traceback - the path through the states based on 
    # our emissions. (Think of that rectangle chart used in Week 1, where we have some
    # probability for each position.).
    traceback_matrix = numpy.zeros((num_states,DNA_length), dtype=int)
    
    # Because theres one fewer transition than the length of the DNA_seq, we don't care
    # about the first column of the traceback array, so we set it as an empty value 
    traceback_matrix[:,0] = numpy.nan

    # We're going to create a probability matrix of the same size.
    probability_matrix = numpy.zeros((num_states,DNA_length))

    # Compute the probability and traceback matrices.
    # `position` refers to each step, as we go from the beginning of `DNA_length` to the end. 
    for position in range(DNA_length):
        # Name the variable `nucleotide` to be the value at the given position in our 
        # encoded DNA sequence.
        nucleotide = encoded_DNA_seq[position]
        # If we're at the first position, we don't care about transition probability, but instead the start prob 
        if position == 0:
            # For each state in the series of states
            for state in range(num_states):
                # For the state, fill in our empty probability_matrix in the correct cell with 
                # the correct probability: the probability of starting in that state times
                # the probability of emitting that nucleotide if in that state.
                probability_matrix[state,position] = s_probs[state] * e_probs[state,nucleotide]
        # If we're at any position besides the first, we care about the transition probability 
        else:
            # Consider the current state, 
            for current_state in range(num_states):
                # We want to find the maximum probability transition from all possible
                # previous states to the current state. We're going to set some
                # temporary variables that we can fill with the highest probability previous state
                # (and it's probability) as we check all previous states
                max_previous_state = None
                max_probability = None
                # We walk through all the states, as these are the possible previous states
                for previous_state in range(num_states):
                    # Compute the probability of the path of interest: the probability from the previous cell times
                    # the probability of getting to current state from this previous state
                    # times the emission probability of the nucleotide in the current state
                    path_prob = probability_matrix[previous_state,position-1] * t_probs[previous_state, current_state] *  e_probs[current_state, nucleotide]
                    # If this path is higher probability than the current max probability path,
                    # (or if we're checking the first possible path)
                    # update our tempory variables with the previous state and the path probability
                    if max_probability == None or path_prob > max_probability:
                        max_previous_state = previous_state
                        max_probability = path_prob
                # Update the probability matrix using the newly computed maximum probability 
                probability_matrix[current_state, position] = max_probability
                # Update the traceback matrix in the proper location to have the index
                # of the max probability previous state
                traceback_matrix[current_state, position] = max_previous_state

    # Now we've filled out both matrices. We first want to find the probability
    # of the max probability path. This is just the max of the last column
    # of the probability matrix
    max_path_probability = numpy.max(probability_matrix[:,-1])
    # Now we find the maximum probability last state as the max probability index
    # of the last column of the probability matrix
    max_end_state = numpy.argmax(probability_matrix[:,-1])

    # Let's set a zero array that will store our maximum state path, as determined
    # from the traceback matrix
    max_path = numpy.zeros(DNA_length, dtype=int)

    # As we navigate the traceback matrix, we're going to update which row (state)
    # we're looking at. We start at the max state of the last column (as determined above)
    current_state = max_end_state
    # We use the `range` command to walk through the traceback matrix. 
    # We go from the last column to the first (zeroth) column, stepping backwards
    # by 1 (i.e. step size -1)
    for i in range(DNA_length-1, -1, -1):
        # we update our max_path array with our current state
        max_path[i] = current_state
        # We now figure out what state we transitioned from, which is going
        # to be the value in the traceback matrix at our current state and
        # current position. We set that as our new current position
        current_state = traceback_matrix[current_state, i]

    # The function the probability of the most likely path, `max_path_probability`,
    # and returns the most likely path itself, `max_path` 
    return max_path_probability, max_path

We check our work by using our test sequence! 

In [ ]:
# We run the Viterbi Algorithm function, using the inputs `start_probs`, `transition_probs`,
# `emission_probs` from our training data and the encoded DNA output from the above function
viterbi_results = viterbi(start_probs, transition_probs, emission_probs, test_seq)

# The Viterbi Algorithm function returns two values: the `max_path_probability` and the 
# `max_path`. We're interested in what that path is - not really its probability. 
# Remember that python starts counting at zero. So since we want the second value returned
# from the Viterbi Algorithm function, `max_path`, we index the `viterbi_results` 
# asking for the [1] term. 
print(viterbi_results[1])